In [1]:
from flask import Flask, render_template, request
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import Keys
from selenium.webdriver.common.by import By
import os
import pandas as pd

In [2]:
driver = webdriver.Chrome(keep_alive=True)
driver.set_window_size(1100, 800)
print("Clicker starts")
username = "info@waddystore.com"
password = "Waddy1208"
driver.get('https://admin.shoplineapp.com/admin/waddystore/')
driver.find_element(By.ID, "staff_email").send_keys(username)
driver.find_element(By.ID, "staff_password").send_keys(password)
driver.find_element(By.ID, "reg-submit-button").click()
print("Login sucessful")

C:\Users\prolo\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: DeprecationWarning: keep_alive has been deprecated, please pass in a Service object
  """Entry point for launching an IPython kernel.


Clicker starts
Login sucessful


In [5]:
sku_id = "5fbb779513512a001476f8bc"
print("Now browsing to SKU: " + sku_id)
driver.get("https://admin.shoplineapp.com/admin/waddystore/products/"+sku_id+"/edit")


driver.find_element(By.XPATH,'//*[@id="product_form"]/div[1]/div[3]/ul/li[4]/a').click()
accept_button = driver.find_element(By.XPATH,'//*[@id="productForm-pricing"]/div/div[3]/div[2]/div[1]/div/div[2]/div/div[2]/label/input')
accept_button_classess = accept_button.get_attribute("class")
print(accept_button_classess)
df = pd.DataFrame()
if "ng-touched" in accept_button_classess:
    print(df.loc[df['Product ID (DO NOT EDIT)'] == sku_id])
    accept_button.click()

driver.find_element(By.XPATH,'//*[@id="product_form"]/div[1]/div[1]/div/span[2]/button/span[2]').click()
print("Saved changes, completed")

Now browsing to SKU: 5fbb779513512a001476f8bc
ng-pristine ng-untouched ng-valid
Saved changes, completed


In [39]:
# cross check with stock level
os.chdir("C:/Users/prolo/OneDrive/Documents/Code_Project/Clicker/codespaces-flask/Check")
file_path = os.listdir()[0]
stock_df = pd.read_excel(file_path,index_col=None, header=0)


os.chdir("C:/Users/prolo/OneDrive/Documents/Code_Project/Clicker/codespaces-flask/Update")
for file in os.listdir():
    # Check whether file is in text format or not
    if file.endswith(".xls"):
        file_path = f"{file}" 
        df, havestock_df, preorder_df, cancel_df = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
        df = pd.read_excel(file_path,index_col=None, header=0)
        df = df.drop(index=0)
        preorder_df = df[df['Preorder Product'] == "Y"]
        havestock_df = preorder_df[preorder_df['Quantity (DO NOT EDIT)'] > 0]
        for barcode in havestock_df["Barcode"]:
            check_stock_df = stock_df.loc[stock_df['商品條碼'] == barcode]
            if check_stock_df['預設倉庫'] > 0:
                pass
        


4204   -3
Name: 預設倉庫, dtype: int64
1370    0
Name: 預設倉庫, dtype: int64
